In [1]:
# LangChain components
from langchain_ollama import OllamaLLM  # Tích hợp mô hình Ollama
from langchain.chains.sql_database.query import create_sql_query_chain  # Tạo chuỗi truy vấn SQL
from langchain.prompts import PromptTemplate  # Template cho prompt
from langchain_community.tools import QuerySQLDataBaseTool  # Tool truy vấn DB
from langchain.sql_database import SQLDatabase  # Wrapper cho DB
from langchain_core.output_parsers import StrOutputParser  # Parser output
from langchain_core.runnables import RunnablePassthrough  # Pipeline processing

# SQLModel for ORM
from sqlmodel import Field, SQLModel, create_engine, Session, select

# Caching
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

In [14]:
llm = OllamaLLM(
    base_url = "http://localhost:11434/",
    model = "qwen2.5-coder:0.5b"
)

In [15]:

class Blog(SQLModel, table=True):
    id: int = Field(primary_key=True, index=True)
    title: str
    content: str

file_path = "./db.sqlite3"
engine = create_engine(f"sqlite:///{file_path}")

def create_all_tables():
    SQLModel.metadata.create_all(engine)

d:\Course\ollama-docker-web-application\ollama-docker-web-application\fastapi\venv\Lib\site-packages\sqlmodel\main.py:641: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Blog, and will be replaced in the string-lookup table.
  DeclarativeMeta.__init__(cls, classname, bases, dict_, **kw)


InvalidRequestError: Table 'blog' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [16]:
create_all_tables()

In [17]:
db = SQLDatabase(engine=engine)

In [18]:
print(db.get_table_info(db.get_usable_table_names()))


CREATE TABLE blog (
	id INTEGER NOT NULL, 
	title VARCHAR NOT NULL, 
	content VARCHAR NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows from blog table:
id	title	content

*/


In [19]:
def standardize_query(query: str):
    if "SQLQuery: " in query:
        query = query.split("SQLQuery: ")[1]
    elif "```sql" in query:
        query = query.split("```sql")[1].split("```")[0].strip()
    return query.split(";")[0]



In [20]:
sql_chain = create_sql_query_chain(
    llm=llm,
    db=db
)

In [21]:
llm_chain = (
    RunnablePassthrough.assign(
        question = lambda x: x["question"]
    ).assign(
        title = lambda x: llm.invoke("Tạo ra title từ question: {}".format(x["question"]))
    ).assign(
        content = lambda x: llm.invoke("Tạo content cho question: {}".format(x["question"]))
    ).assign(
        db_exec = lambda x: db.run('''INSERT INTO "blog" ("title", "content") VALUES ('{}', '{}')'''.format(
            x["title"].replace("'", "''"),
            x["content"].replace("'", "''"),
        ))
    )
)

In [22]:
cache = InMemoryCache()
set_llm_cache(cache)

In [23]:
generation_params = {
    "temperature": 0.7,        # Điều chỉnh độ ngẫu nhiên (0-1)
    "top_k": 10,              # Giới hạn từ vựng top K tokens
    "top_p": 0.95,            # Ngưỡng nucleus sampling
    "num_ctx": 2048,          # Kích thước cửa sổ ngữ cảnh
    "num_thread": 1,          # Số luồng sử dụng
    "num_predict": 200,       # Số tokens tối đa dự đoán
    "repeat_last_n": 64,      # Số tokens cuối cùng xem xét lặp lại
    "repeat_penalty": 1.15    # Hệ số phạt lặp lại
}


In [24]:
run_llm = (llm_chain.with_config({
    **generation_params
}).invoke({
    "question": "Thêm bài viết về Typescript"
}))



In [25]:
print('''
hello
hello
hello
hello
hello
hello

''')


hello
hello
hello
hello
hello
hello


